In [10]:
import zipfile

with zipfile.ZipFile('BC NEW dataset (40X).zip','r') as zip_ref:
    zip_ref.extractall('/content/')

In [31]:
import os

def count_images_in_folder(folder_path):
    """Counts the number of images in a given folder.

    Args:
        folder_path (str): The path to the folder.

    Returns:
        int: The number of images in the folder.
    """

    image_extensions = ['.jpg', '.jpeg', '.png', '.gif']  # Add more extensions if needed
    image_count = 0

    for filename in os.listdir(folder_path):
        if any(filename.endswith(ext) for ext in image_extensions):
            image_count += 1

    return image_count

# Replace 'your_folder_path' with the actual path to your folder
folder_path = "C:/40xhighdensity/Benign"
#folder_path ="C:/benignhigh400x"
#folder_path = "C:/benignmedium400x"
#folder_path = "C:/benign400Mx"
#folder_path = "C:/content/BC NEW dataset (400X)/maligant"
total_images = count_images_in_folder(folder_path)
print("Total images in the folder:", total_images)

Total images in the folder: 1548


In [1]:
#40x calculation of density
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from skimage.transform import resize

# Load pre-trained Attention U-Net model (ensure you have the correct path)
MODEL_PATH = "deep_unet_attention_model.h5"
unet_model = load_model(MODEL_PATH, compile=False)

def segment_nuclei(image):
    """
    Segment nuclei using a deep U-Net model with attention mechanisms.
    """
    image_resized = resize(image, (256, 256), preserve_range=True, anti_aliasing=True).astype(np.float32)
    image_resized = np.expand_dims(image_resized, axis=0)  # Add batch dimension
    prediction = unet_model.predict(image_resized)
    mask = prediction[0, :, :, 0]  # Extract single-channel mask
    mask = (mask > 0.5).astype(np.uint8) * 255  # Apply thresholding
    return cv2.resize(mask, (image.shape[1], image.shape[0]))

def calculate_density(image, microns_per_pixel_x, microns_per_pixel_y):
    """
    Calculate nuclei density using deep learning segmentation.
    """
    binary_mask = segment_nuclei(image)
    num_labels, _ = cv2.connectedComponents(binary_mask)
    num_nuclei = num_labels - 1
    height, width = binary_mask.shape
    area_sq_microns = height * microns_per_pixel_y * width * microns_per_pixel_x
    area_sq_mm = area_sq_microns / 1e6
    density_per_mm2 = num_nuclei / area_sq_mm
    return num_nuclei, density_per_mm2, binary_mask

def split_image_into_patches(image, patch_size):
    """
    Split the image into overlapping patches.
    """
    patches = []
    height, width = image.shape[:2]
    for y in range(0, height, patch_size // 2):  # 50% overlap
        for x in range(0, width, patch_size // 2):
            patch = image[y:y + patch_size, x:x + patch_size]
            if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
                patches.append(patch)
    return patches

def process_images_in_folder(folder_path, microns_per_pixel_x, microns_per_pixel_y, patch_size):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    all_density_results = []

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        patches = split_image_into_patches(image, patch_size)
        
        for idx, patch in enumerate(patches):
            num_nuclei, density_mm2, binary_mask = calculate_density(patch, microns_per_pixel_x, microns_per_pixel_y)
            all_density_results.append({
                'image_file': image_file,
                'patch_index': idx,
                'num_nuclei': num_nuclei,
                'density_per_mm2': density_mm2
            })
            density_folder = "Low Density" if density_mm2 < 300 else "Medium Density" if density_mm2 < 700 else "High Density"
            density_path = os.path.join("nuclei_density_results40xd", density_folder)
            os.makedirs(density_path, exist_ok=True)
            cv2.imwrite(os.path.join(density_path, f"{image_file}_patch_{idx + 1}.png"), binary_mask)
            print(f"Image: {image_file}, Patch {idx + 1}: {num_nuclei} nuclei, Density: {density_mm2:.2f} nuclei/mm²")

    df_density = pd.DataFrame(all_density_results)
    os.makedirs("nuclei_density_results40xd", exist_ok=True)
    df_density.to_csv("nuclei_density_results40xd/combined_density_results40xd.csv", index=False)
    print("Results saved.")

# Example usage
folder_path = "E:/BC NEW dataset (40X)/maligant 40X"
MICRONS_PER_PIXEL_X = 0.1625
MICRONS_PER_PIXEL_Y = 0.1625
PATCH_SIZE = 224  # Patch size (224x224 pixels)
process_images_in_folder(folder_path, MICRONS_PER_PIXEL_X, MICRONS_PER_PIXEL_Y, PATCH_SIZE)


Image: SOB_M_DC-14-16716-40-001.png, Patch 1:
Number of nuclei: 8
Nuclei density: 60.38 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-16716-40-001.png, Patch 2:
Number of nuclei: 6
Nuclei density: 45.28 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-16716-40-001.png, Patch 3:
Number of nuclei: 14
Nuclei density: 105.66 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-16716-40-001.png, Patch 4:
Number of nuclei: 7
Nuclei density: 52.83 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-16716-40-001.png, Patch 5:
Number of nuclei: 10
Nuclei density: 75.47 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-16716-40-001.png, Patch 6:
Number of nuclei: 9
Nuclei density: 67.93 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-16716-40-002.png, Patch 1:
Number of nuclei: 20
Nuclei density: 150.95 nuclei per mm²
------------------------------
Image: SOB_M_DC-14-16716-40-002.png, Patch 2:
Number of n

In [16]:
import os

def count_images(folder_path):
  """Counts the number of image files in a given folder.

  Args:
    folder_path: The path to the folder containing the images.

  Returns:
    The number of image files in the folder.
  """

  image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add other image extensions if needed
  image_count = 0

  for file_name in os.listdir(folder_path):
    file_extension = os.path.splitext(file_name)[1]
    if file_extension.lower() in image_extensions:
      image_count += 1

  return image_count

# Replace 'folder1', 'folder2', and 'folder3' with the actual paths to your folders
folder1_path = 'nuclei_density_results40xd/Low Density'
folder2_path = 'nuclei_density_results40xd/Medium Density'
folder3_path = 'nuclei_density_results40xd/High Density'

image_count_folder1 = count_images(folder1_path)
image_count_folder2 = count_images(folder2_path)
image_count_folder3 = count_images(folder3_path)

print("Number of images in folder 1:", image_count_folder1)
print("Number of images in folder 2:", image_count_folder2)
print("Number of images in folder 3:", image_count_folder3)

Number of images in folder 1: 2794
Number of images in folder 2: 1527
Number of images in folder 3: 1223
